## DEMO

In [1]:
# CELL 1: Install packages
!pip install -q \
    transformers \
    torch \
    faiss-gpu \
    sentence-transformers \
    beautifulsoup4 \
    tqdm \
    unidecode \
    pandas \
    numpy \
    scikit-learn \
    gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 28.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.4/136.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip -q install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.2 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip -q install flask-ngrok

In [4]:
!pip show flask-ngrok

Name: flask-ngrok
Version: 0.0.25
Summary: A simple way to demo Flask apps from your machine.
Home-page: https://github.com/gstaff/flask-ngrok
Author: Grant Stafford
Author-email: 
License: UNKNOWN
Location: /usr/local/lib/python3.10/dist-packages
Requires: Flask, requests
Required-by: 


In [5]:
!pip -q install pyngrok

In [6]:
# ==========================================================
# CELL 13.A: XÁC THỰC NGROK (CHẠY 1 LẦN)
# ==========================================================
import os
from pyngrok import ngrok

# THAY THẾ BẰNG AUTHTOKEN CỦA BẠN
NGROK_AUTHTOKEN = "33xssoeiIZT6i4jrZb0ZAOECVz6_7garq9w8GQ8kri2Ni92J" 

# Kiểm tra để tránh chạy lại không cần thiết
if "NGROK_AUTHTOKEN" not in os.environ:
    os.environ["NGROK_AUTHTOKEN"] = NGROK_AUTHTOKEN
    try:
        ngrok.set_auth_token(NGROK_AUTHTOKEN)
        print("✅ Ngrok Authtoken đã được set.")
    except Exception as e:
        print(f"Lỗi khi set Ngrok Authtoken: {e}")
else:
    print("Ngrok Authtoken đã được set từ trước.")

✅ Ngrok Authtoken đã được set.                                                                      


In [ ]:
# ==========================================================
# CELL 13.B: DEMO VỚI FLASK + PYNGROK (SỬA ĐỂ ĐỌC .pkl)
# ==========================================================
import os
import re
import pandas as pd
import faiss
import torch
import json
import sys
import pickle # <-- 1. Thêm thư viện pickle
from flask import Flask, request, render_template_string, jsonify
from pyngrok import ngrok
from transformers import pipeline, AutoTokenizer
from sentence_transformers import SentenceTransformer, CrossEncoder
from tqdm import tqdm
import time

print("Bắt đầu... (Quá trình này sẽ tải model và có thể mất vài phút)")

# ==========================================================
# Bước 2: Định nghĩa cấu hình
# ==========================================================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BI_ENCODER_MODEL = "bkai-foundation-models/vietnamese-bi-encoder"
CROSS_ENCODER_MODEL = "namdp-ptit/ViRanker"
LLM_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
K_RETRIEVE = 25
TOPK_FINAL = 3
RERANK_THRESHOLD = 0.5

# === SỬA LỖI Ở ĐÂY ===
# 2. Sửa đường dẫn trỏ đến file .pkl và .index của bạn
INDEX_PATH = "/kaggle/input/seg-5k-crawl/vinmec_faiss_token_chunk.index" 
META_PATH = "/kaggle/input/seg-5k-crawl/vinmec_chunks.pkl"
# ======================

# ==========================================================
# Bước 3: Tải Model và Dữ liệu (chỉ tải 1 lần)
# ==========================================================
print(f"Đang tải model trên {DEVICE}...")
try:
    bi_encoder = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)
    cross_encoder = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)
    llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
    generator = pipeline(
        "text-generation",
        model=LLM_MODEL,
        tokenizer=llm_tokenizer,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map=DEVICE,
        pad_token_id=llm_tokenizer.eos_token_id,
        eos_token_id=llm_tokenizer.eos_token_id,
        max_new_tokens=512
        #"load_in_8bit": True
        
        
    )
    print("...Tải xong 3 model.")

    # === SỬA LỖI QUAN TRỌNG Ở ĐÂY ===
    print("Loading FAISS index and metadata...")
    if not os.path.exists(INDEX_PATH) or not os.path.exists(META_PATH):
        raise FileNotFoundError(
            f"LỖI: Không tìm thấy {INDEX_PATH} hoặc {META_PATH}. "
            "Hãy kiểm tra lại đường dẫn trong 'Add Data' (Input)."
        )
    
    print(f"Đang tải index: {INDEX_PATH}")
    index = faiss.read_index(INDEX_PATH)
    
    # 3. Sửa logic tải file: Dùng pickle.load()
    print(f"Đang tải file meta: {META_PATH}...")
    with open(META_PATH, 'rb') as f:
        meta_data_from_pkl = pickle.load(f)

    # Kiểm tra xem file pkl là DataFrame hay list[dict]
    # (Code Flask của chúng ta cần meta là list[dict])
    if isinstance(meta_data_from_pkl, pd.DataFrame):
        print("Phát hiện DataFrame -> Đang chuyển sang list[dict]...")
        meta = meta_data_from_pkl.to_dict("records")
    elif isinstance(meta_data_from_pkl, list):
        print("Phát hiện list[dict] (Tốt!).")
        meta = meta_data_from_pkl
    else:
        raise TypeError("File .pkl không chứa DataFrame hoặc list[dict] như mong đợi.")
    
    print(f"✅ Tải xong {len(meta)} chunk metadata.")
    
except Exception as e:
    print(f"LỖI KHỞI TẠO: {e}")
    sys.exit(1)


# ==========================================================
# Bước 4: Định nghĩa các hàm RAG (ĐÃ THÊM THRESHOLD)
# ==========================================================
RAG_PROMPT_TEMPLATE = """<|im_start|>system
Bạn là một trợ lý AI y tế hữu ích. Nhiệm vụ của bạn là trả lời câu hỏi của người dùng dựa trên thông tin ngữ cảnh được cung cấp.
Hãy trả lời một cách súc tích, chính xác, chỉ dựa vào ngữ cảnh. Nếu ngữ cảnh không chứa thông tin để trả lời hoặc người dùng hỏi không liên quan đến y tế, hãy nói: "Xin lỗi, tôi không tìm thấy thông tin phù hợp trong tài liệu."
Không được bịa đặt thông tin.

NGỮ CẢNH:
{context}
<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant
"""
@torch.no_grad()
def search_retrieval(query, k_retrieve=K_RETRIEVE):
    # (Hàm này giữ nguyên, không thay đổi)
    q_vec = bi_encoder.encode([query], normalize_embeddings=True, device=DEVICE)
    D, I = index.search(q_vec, k_retrieve)
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1: continue
        meta_copy = meta[idx].copy()
        meta_copy['bi_score'] = float(score)
        results.append(meta_copy)
    return results
@torch.no_grad()
def rerank_hits(query, hits):
    # (Hàm này giữ nguyên, không thay đổi)
    if not hits: return []
    pairs = [(query, h['text']) for h in hits]
    scores = cross_encoder.predict(pairs)
    for h, score in zip(hits, scores):
        h['rerank_score'] = float(score)
    return sorted(hits, key=lambda x: x['rerank_score'], reverse=True)

# === HÀM NÀY ĐƯỢC CẬP NHẬT ===
def rag_answer(query, topk=TOPK_FINAL):
    retrieved_hits = search_retrieval(query, k_retrieve=K_RETRIEVE)
    reranked_hits = rerank_hits(query, retrieved_hits)
    
    # Lấy Top-K kết quả ban đầu
    initial_top_k = reranked_hits[:topk]
    
    if not initial_top_k:
        return "Xin lỗi, tôi không tìm thấy thông tin phù hợp trong tài liệu.", []

    # <-- THÊM MỚI: LỌC KẾT QUẢ BẰNG THRESHOLD -->
    # Chỉ giữ lại các hits có điểm rerank >= ngưỡng
    top_k_hits = [h for h in initial_top_k if h['rerank_score'] >= RERANK_THRESHOLD]
    
    # Nếu (sau khi lọc) không còn gì, nghĩa là các kết quả đều kém
    if not top_k_hits:
        # In ra log để kiểm tra (bạn có thể thấy trong console của Kaggle)
        print(f"Ngưỡng an toàn: Kết quả top 1 có score ({initial_top_k[0]['rerank_score']:.4f}) < {RERANK_THRESHOLD}. Từ chối trả lời.")
        return "Xin lỗi, tôi không tìm thấy thông tin phù hợp trong tài liệu.", []
    # ============================================

    # Chỉ những hits "sạch" (đã qua lọc) mới được đưa vào context
    context = "\n\n---\n\n".join([h['text'] for h in top_k_hits])
    prompt = RAG_PROMPT_TEMPLATE.format(context=context, question=query)
    
    outputs = generator(prompt)
    answer = outputs[0]['generated_text'].split("<|im_start|>assistant\n")[-1].strip()
    answer = answer.replace("<|im_end|>", "").strip()
    
    return answer, top_k_hits

# ==========================================================
# Bước 5: Khởi tạo Web App (Flask) (KHÔNG THAY ĐỔI)
# ==========================================================
app = Flask(__name__)
HTML_TEMPLATE = """
<!doctype html>
<html lang="vi">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale-1">
    <title>Demo RAG Y Tế</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif; margin: 2em; background: #f9f9f9; color: #333; }
        .container { max-width: 800px; margin: auto; background: white; padding: 25px; border-radius: 12px; box-shadow: 0 4px 12px rgba(0,0,0,0.05); }
        h1 { color: #0056b3; text-align: center; }
        form { display: flex; flex-direction: column; }
        textarea { width: 97%; padding: 12px; font-size: 16px; border-radius: 8px; border: 1px solid #ccc; margin-bottom: 15px; font-family: inherit; }
        input[type="submit"] { background: #007bff; color: white; padding: 12px 25px; border: none; border-radius: 8px; cursor: pointer; font-size: 16px; font-weight: bold; transition: background 0.2s; }
        input[type="submit"]:hover { background: #0056b3; }
        .result { margin-top: 30px; border-top: 1px solid #eee; padding-top: 30px; }
        .answer { background: #eef7ff; padding: 20px; border-radius: 8px; white-space: pre-wrap; font-size: 1.05em; line-height: 1.6; border-left: 5px solid #007bff; }
        .sources { margin-top: 25px; }
        h2 { color: #333; border-bottom: 2px solid #eee; padding-bottom: 5px; }
        .source-item { margin-bottom: 12px; padding: 15px; background: #fafafa; border: 1px solid #ddd; border-radius: 8px; }
        .source-item .title { font-weight: 600; color: #0056b3; }
        .source-item .url { font-size: 0.9em; color: #555; margin-top: 5px; }
        .source-item .score { font-size: 0.8em; color: #777; margin-top: 8px; }
        .loading { display: none; text-align: center; font-size: 1.2em; color: #555; margin-top: 20px; }
        
        /* <-- THÊM CSS Ở ĐÂY --> */
        .response-time {
            text-align: center;
            font-size: 0.95em;
            color: #555;
            margin-bottom: 20px;
            font-style: italic;
        }
    </style>
    <script>
        function handleSubmit() {
            document.getElementById('submit-btn').disabled = true;
            document.getElementById('submit-btn').value = 'Đang xử lý...';
            document.getElementById('loading-indicator').style.display = 'block';
            return true;
        }
    </script>
</head>
<body>
    <div class="container">
        <h1>Chatbot Về Y Tế cho Mẹ và Bé</h1>
        <form action="/ask" method="POST" onsubmit="return handleSubmit();">
            <label for="query"><b>Đặt câu hỏi:</b></label>
            <textarea id="query" name="query" rows="3" required>{{ query | safe }}</textarea>
            <input id="submit-btn" type="submit" value="Hỏi">
        </form>
        <div id="loading-indicator" class="loading">
            <p>Đang tìm kiếm và tạo câu trả lời... (có thể mất 10-30 giây)</p>
        </div>
        
        {% if answer %}
        <div class="result">
            
            {% if time_taken %}
            <div class="response-time">
                ⏱️ Thời gian phản hồi: {{ "%.2f"|format(time_taken) }} giây
            </div>
            {% endif %}
            <h2>Câu trả lời:</h2>
            <div class="answer">{{ answer | safe }}</div>
            <h2>Nguồn tham khảo (Top {{ sources | length }}):</h2>
            <div class="sources">
                {% for source in sources %}
                <div class="source-item">
                    <div class="title">{{ source['title'] | default('(Không có tiêu đề)', true) }}</div>
                    <div class="url">{{ source['url'] }}</div>
                    <div class="score">(Re-rank Score: {{ "%.4f"|format(source['rerank_score']) }})</div>
                </div>
                {% endfor %}
            </div>
        </div>
        {% endif %}
    </div>
</body>
</html>
"""


# ==========================================================
# Bước 6: Định nghĩa các Route (ĐÃ THÊM ĐO THỜI GIAN)
# ==========================================================
@app.route('/')
def home():
    # Thêm time_taken=None
    return render_template_string(HTML_TEMPLATE, query="", answer=None, sources=[], time_taken=None)

@app.route('/ask', methods=['POST'])
def ask():
    start_time = time.time() # <-- BẮT ĐẦU ĐO THỜI GIAN
    try:
        query = request.form['query']
        answer, sources = rag_answer(query, topk=TOPK_FINAL)
        
        end_time = time.time() # <-- KẾT THÚC ĐO THỜI GIAN
        elapsed_time = end_time - start_time
        
        return render_template_string(
            HTML_TEMPLATE, 
            query=query, 
            answer=answer, 
            sources=sources, 
            time_taken=elapsed_time # <-- Truyền thời gian vào template
        )
    except Exception as e:
        print(f"Lỗi: {e}")
        end_time = time.time() # <-- Vẫn tính thời gian ngay cả khi lỗi
        elapsed_time = end_time - start_time
        
        return render_template_string(
            HTML_TEMPLATE, 
            query=query, 
            answer=f"Đã xảy ra lỗi: {e}", 
            sources=[],
            time_taken=elapsed_time # <-- Truyền thời gian vào template
        )

# ==========================================================
# Bước 7: Chạy App (KHÔNG THAY ĐỔI)
# ==========================================================
if __name__ == '__main__':
    # Mở một tunnel công khai sử dụng ngrok, trỏ đến port 5000 của Flask
    listener = ngrok.connect(5000) 
    
    print(f"\n✅ Tải xong! Khởi động server Flask...")
    
    # === SỬA LỖI Ở ĐÂY ===
    # Thuộc tính đúng là .public_url (không có dấu ngoặc '()')
    print(f"👉 Demo đang chạy tại: {listener.public_url}") 
    # ====================
    
    # Chạy app Flask trên port 5000
    app.run(port=5000)

Bắt đầu... (Quá trình này sẽ tải model và có thể mất vài phút)
Đang tải model trên cuda...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]